# Framework v4.5 — Capacity → Dimension Shift

**Capacity-only falsifiable demo**: the same fixed substrate yields different effective geometry
(spectral dimension) when only the observational capacity `C_geo` changes.

Everything else is held fixed: lattice dimension D, side length N, lattice spacing a=1,
diffusion convention, σ grid, derivative estimator, and plateau window.

In [ ]:
# Setup — install dependencies if needed (e.g., in Colab)
import sys, os
try:
    import numpy as np
    import matplotlib.pyplot as plt
except ImportError:
    !pip install numpy matplotlib
    import numpy as np
    import matplotlib.pyplot as plt

# Add parent dir to path so dimshift is importable
sys.path.insert(0, os.path.abspath('..'))
# If running from the notebook dir itself:
if not os.path.exists('../dimshift'):
    sys.path.insert(0, os.path.abspath('.'))

In [ ]:
from dimshift import SweepConfig, run_capacity_sweep
from dimshift.plotting import plot_heatmap, plot_representative_curves, plot_phase_diagram
print('dimshift library loaded successfully')

## 1. Configure and Run the Experiment

All parameters are fixed except `C_geo`. The sweep tests `C_geo` from 0.05 to 1.0.

In [ ]:
config = SweepConfig(
    D=3,            # 3D cubic lattice
    N=64,           # 64 sites per dimension (64^3 = 262,144 total)
    C_geo_steps=30, # 30 capacity steps
    n_sigma=400,    # 400 diffusion time points
)

print(f'Lattice: {config.D}D, N={config.N}, total sites = {config.N**config.D:,}')
print(f'C_geo grid: [{config.C_geo_min}, {config.C_geo_max}] x {config.C_geo_steps} steps')
print(f'σ grid: [{config.sigma_min}, {config.sigma_max}] x {config.n_sigma} points')
lo, hi = config.plateau_window()
print(f'Plateau window: σ ∈ [{lo:.1f}, {hi:.1f}]')

In [ ]:
result = run_capacity_sweep(config)
print(f'Sweep completed in {result.elapsed_s:.2f}s')
print(f'Run ID: {result.run_id}')
print(f'\nDetected thresholds:')
for t in result.thresholds:
    print(f'  d_s = {t["target_dimension"]:.1f}  at  C_geo = {t["C_geo_threshold"]:.4f}')

## 2. Validation: Full Capacity Check

At `C_geo = 1`, all dimension weights = 1 and `d_s` should equal `D` exactly (within ~1%).

In [ ]:
ds_full = result.ds_plateau[-1]
D = config.D
error_pct = abs(ds_full - D) / D * 100
print(f'd_s(C_geo=1) = {ds_full:.4f}')
print(f'Expected: {D}.0000')
print(f'Error: {error_pct:.2f}%')
print(f'Status: {"PASS" if error_pct < 1.0 else "WARN"}')

## 3. Heatmap — The Money Plot

Shows `d_s(σ)` as a function of both `C_geo` (x-axis) and `log₁₀(σ)` (y-axis).
Horizontal bands of uniform colour at integer dimensions demonstrate the capacity → geometry effect.

In [ ]:
fig = plot_heatmap(result)
plt.show()

## 4. Representative Curves

`d_s(σ)` vs `log₁₀(σ)` for C_geo values near the activation thresholds `k/D`.
Each curve plateaus at the expected integer dimension in the validation window.

In [ ]:
fig = plot_representative_curves(result)
plt.show()

## 5. Phase Diagram: d_eff vs C_geo

The measured plateau spectral dimension as a function of capacity.
Shows the staircase structure: d_s ≈ 1 → 2 → 3 at the predicted thresholds.

In [ ]:
fig = plot_phase_diagram(result)
plt.show()

## 6. Capacity Weights Detail

Visualise how the per-dimension weights w_d evolve with C_geo.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
C_vals = result.C_geo_values
for d in range(config.D):
    w_d = [result.weights_list[i][d] for i in range(len(C_vals))]
    ax.plot(C_vals, w_d, lw=2, label=f'$w_{d+1}$ (dim {d+1})')
ax.set_xlabel('$C_{\\mathrm{geo}}$')
ax.set_ylabel('Weight $w_d$')
ax.set_title('Per-Dimension Capacity Weights')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Summary Table

In [ ]:
print(f'{"C_geo":>8} {"d_eff_nom":>10} {"d_s_plat":>10} {"weights":>30}')
print('-' * 62)
step = max(1, len(result.C_geo_values) // 10)
for i in range(0, len(result.C_geo_values), step):
    c = result.C_geo_values[i]
    d_nom = result.d_eff_nominal[i]
    d_s = result.ds_plateau[i]
    w = result.weights_list[i]
    w_str = '[' + ', '.join(f'{x:.2f}' for x in w) + ']'
    print(f'{c:8.3f} {d_nom:10.3f} {d_s:10.3f} {w_str:>30}')

## 8. Save Artifacts

Optionally save all outputs (CSV, JSON, plots) to disk.

In [ ]:
from dimshift.sweep import write_artifacts
from dimshift.plotting import save_all_figures

out_dir = f'../outputs/capacity_dimshift/{result.run_id}'
artifact_paths = write_artifacts(result, out_dir)
fig_paths = save_all_figures(result, out_dir)
print('Artifacts saved:')
for name, p in {**artifact_paths, **fig_paths}.items():
    print(f'  {name}: {p}')